In [1]:
'''
This script hooks a layer from a loaded trained SchNet checkpoint for a forward pass
of the values in that layer
'''

import torch
import schnetpack as spk
import math

import numpy as np
from numpy import savetxt

# Define important functions

def interactions_hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global interactions_output
    #Update the interactions_output variable to be equal to our output tensor
    interactions_output=out_tensor 

def print_atom_coordinates(idx,props):
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()    
    # print molecule number in set, idx, and atom coordinates for molecule identification
    print(idx)
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 
    print('')
    
def get_number_of_atoms(idx,props):
    z = props['_positions'][ :,2]
    number_of_atoms=len(z)
    return number_of_atoms
    

In [2]:
# Initiate 
# a) QM9 dataset and  
# b) SchNet model architechure 
# as it is exactly saved checkpoint file that will be loaded later

# a) Load QM9 dataset
from schnetpack.datasets import QM9
qm9data = QM9('./qm9.db', remove_uncharacterized=True)

# split the data set according to saved split file (after training)
train, val, test = spk.data.train_test_split(qm9data, split_file='../data/trained_models/qm9_i6_30f/split.npz')
train_loader = spk.AtomsLoader(train, batch_size=100, shuffle=True)

# load the test, train and validation dataset loaders
test_loader = spk.AtomsLoader(test, batch_size=100)

# load some atom reference data that were used in inititial training conditions
atomrefs = qm9data.get_atomref(QM9.U0)

# get statistics on trained dataset
means, stddevs = train_loader.get_statistics(
    QM9.U0, divide_by_atoms=True, single_atom_ref=atomrefs)

# b) load SchNet representation model as it is exactly saved in checkpoint file
schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=5,
    cutoff=4., cutoff_network=spk.nn.cutoff.CosineCutoff
)
output_U0 = spk.atomistic.Atomwise(n_in=30, atomref=atomrefs[QM9.U0], property=QM9.U0,
                                   mean=means[QM9.U0], contributions=True, stddev=stddevs[QM9.U0])


# load atomistic model
model = spk.AtomisticModel(representation=schnet, output_modules=output_U0)


/home/amerelsamman/Programs/anaconda3/lib/python3.7/site-packages/schnetpack/data/atoms.py:327: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  properties[pname] = torch.FloatTensor(prop)
/home/amerelsamman/Programs/anaconda3/lib/python3.7/site-packages/ase/atoms.py:972: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


In [3]:
# Load saved checkpoint file of model
checkpoint_path = '../data/trained_models/qm9_i6_30f/qm9_i6_30f_20g-1000-500-4_300.pth'
load_checkpoint = torch.load(checkpoint_path)


# load state dictionary from loaded checkpoint
model.load_state_dict(load_checkpoint)


#Hook our function to the interaction layer during the forward pass
model.representation.interactions[0].register_forward_hook(interactions_hook)


In [4]:

#set up device that will be used in forward pass and atoms converter for input
device='cpu'

#load the test dataset
test_loader = spk.AtomsLoader(test, batch_size=100)

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)


In [5]:
# how many inputs do you want to evaluate?
print('how many molecules do you want to evaluate?')
number_of_inputs = input()
number_of_inputs = int(number_of_inputs)

#initiate data to hold specified number of inputs X 30 for dimension of trained layer
data = np.zeros((1,30))
index=0


# for a number of inputs, run a forward pass on interesting trained layers of SchNet model
for idx in range(number_of_inputs):
    
    #load atomistic data and properties from qm9data
    at, props = qm9data.get_properties(idx)
    
    #print molecule number in dataset and atom coordinates
    print_atom_coordinates(idx,props)

    # set calculator to atomistic data
    at.set_calculator(calculator)
    
    # convert atomistic data to inputs for the model
    inputs = converter(at)

    #We choose None to instatiate the variable originally
    interactions_output=None

    #Forward pass of the tensor inputs
    model(inputs)
    
    number_of_atoms = get_number_of_atoms(idx,props)
    
    #reshape interaction tensor to a 2D array
    interactions_output = interactions_output.reshape(number_of_atoms,30)
    
    #store data for each oxygen found
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                data[index,j] = interactions_output[place,j]
            index = index+1
            data = np.vstack((data,np.zeros(30)))
            
#save to csv
savetxt('../data/data.csv',data[ : (len(data)-1)],delimiter=',')

            



how many molecules do you want to evaluate?
3000
0
C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613

1
N -0.03160822 0.021159047 0.060332563
H 0.026075294 -0.9904034 -0.029608395
H 0.9246072 0.35579658 -0.030988995
H -0.5114599 0.34058344 -0.77777386

2
O -0.056248687 0.035458494 0.0007260183
H 0.042878296 -0.9215089 -0.00534094
H 0.84990215 0.35871136 -0.0061824406

3
C 0.5995395 0.0 0.0
C -0.5995395 0.0 0.0
H -1.6616386 0.0 0.0
H 1.6616386 0.0 0.0

4
C -0.0075630867 0.5570853 0.003070365
N 0.008071566 -0.59453905 -0.0032767907
H -0.022041854 1.6235689 0.008948269

5
C -0.008137181 0.5992856 0.002790173
O 0.0081545 -0.60058945 -0.0028012523
H 0.9208008 1.2085854 -0.0010057696
H -0.9532692 1.1830921 0.012220519

6
C -0.010403622 0.7647314 0.004216852
C 0.010404125 -0.7647325 -0.004217135
H 1.0031731 1.1788927 -0.0032747546
H -0.53377575 1.1627601

H -1.2028258 1.2264612 -1.087867
H 1.2586594 0.73993236 -1.117245
H 1.2980903 1.4860449 0.49100813
H 1.4190933 -1.4533845 0.09110165
H 0.70723516 -0.71630615 1.5384057
H -1.5143578 -1.2591239 0.5057526
H -0.7761558 -1.2462053 -1.106577

47
C -0.7670668 0.80218834 -0.18759951
C 0.761323 0.7041908 -0.001375825
O 0.69508994 -0.7269163 0.17002626
C -0.73851645 -0.72804123 0.0069237044
H -1.297961 1.3626112 0.5833927
H -1.0984524 1.1435276 -1.1694459
H 1.3681483 0.98485094 -0.8715061
H 1.168062 1.2046599 0.88644487
H -1.2520474 -1.1063579 0.8998369
H -1.0518695 -1.3260804 -0.8581154

48
C -0.112086184 1.4519535 0.4722047
C -0.0147302225 -0.050548814 0.4463064
C -0.016417842 -0.8160251 1.7378886
N 0.07233551 -0.7391456 -0.62733203
O 0.06485818 0.08503627 -1.7684115
H 0.7738884 1.8997909 0.0100006135
H -0.9710368 1.7890232 -0.116678566
H -0.20807421 1.8275121 1.4928572
H -0.93573266 -0.6200971 2.303366
H 0.06302906 -1.8877655 1.5487098
H 0.81983924 -0.5026511 2.374872
H 0.13022676 -0.5596606 

85
N -0.057123955 1.7483617 0.49261686
C -0.051818028 0.54126346 0.15909214
C -1.1001686 -0.5495181 0.060917392
C 0.08602445 -1.4154221 -0.40646124
O 0.9838027 -0.27093995 -0.2644824
H 0.87668735 2.15694 0.42496663
H -1.5559136 -0.8302318 1.0115194
H -1.8817728 -0.36501205 -0.6774614
H 0.38472512 -2.2301471 0.25738493
H 0.05677837 -1.7620572 -1.4421631

86
O 0.018968131 -1.7640274 -0.34606293
C 0.00592748 -0.58591306 -0.14286245
C -1.1154436 0.44729778 0.07466429
C -0.015023246 1.5084827 0.3856067
C 1.1046822 0.47698423 0.045027785
H -1.6964426 0.6352329 -0.8339263
H -1.8139421 0.20165703 0.8795269
H -0.03536181 2.3933592 -0.25162515
H -0.0054291748 1.831497 1.4278737
H 1.8307295 0.2503917 0.8308742
H 1.6559936 0.68006015 -0.87867796

87
O 0.056591343 -1.7825387 -0.019702515
C 0.0003466606 -0.5835165 -0.006592764
C -1.1206632 0.4892062 0.0019281352
C 0.034038052 1.5315369 0.0173767
N 0.96318835 0.39859927 0.0073430813
H -1.7455553 0.48987544 -0.89248526
H -1.7515781 0.46973193 0.891878

108
N 1.4244626 0.9579274 -0.008468414
C 1.2918477 -0.30065817 -0.013436042
N 0.060311563 -0.95755625 -0.0057856534
C -1.2217767 -0.4254112 0.009063386
O -1.5203862 0.73662335 0.018326586
H 2.4145567 1.1987293 -0.016082177
H 2.1115346 -1.0306176 -0.024377037
H 0.10537184 -1.9650236 -0.011645644
H -1.9669752 -1.2483422 0.011400136

109
N 0.7839506 2.1481187 0.004280184
C 0.85980904 0.89550203 -0.0035450475
O -0.297446 0.1635308 0.003514263
C -0.19885314 -1.1969259 -0.0045071156
O -1.15787 -1.8988776 0.00095024036
H 1.7159578 2.5592022 -0.0023584922
H 1.766076 0.27907202 -0.015709855
H 0.84739363 -1.5530415 -0.016322123

110
O -1.4572579 -0.8431472 0.008857734
C -1.2600759 0.33831686 0.012946789
N -0.013109395 0.96676713 0.005070856
C 1.2504364 0.3724739 -0.009277258
O 1.4795827 -0.80318785 -0.017458055
H -2.068963 1.0969422 0.02400012
H -0.026811669 1.9773167 0.010396228
H 2.038459 1.1528236 -0.012080559

111
C -0.03515972 2.584621 0.01434152
C -0.015212007 1.133484 0.006053129
C 0.0012

146
C -0.52654064 1.8618008 -0.8486309
C -0.874241 0.38031533 -0.6622697
C -0.19642977 -0.21688163 0.546739
C 0.3078578 -1.5945843 0.5743214
O 1.2034097 -0.4914797 0.4231205
H 0.5541094 1.9897314 -0.9610879
H -1.0140116 2.2738636 -1.7369565
H -0.84683967 2.455929 0.014316729
H -1.9598095 0.257966 -0.5533584
H -0.57528776 -0.1896832 -1.5493454
H -0.46615562 0.23784547 1.5030855
H 0.37986928 -2.1313202 1.5195807
H 0.19109558 -2.2250423 -0.30680424

147
C -0.40309435 1.9070009 0.74577
C -0.77674586 0.4374234 0.5749955
N -0.08529595 -0.14513314 -0.57154214
C 1.2316492 -0.71442145 -0.30580184
C 0.06003689 -1.594941 -0.5712789
H 0.66762036 2.0182307 0.94605994
H -0.63618815 2.4694943 -0.16267249
H -0.9517315 2.3513412 1.581902
H -0.5596347 -0.117008805 1.5074964
H -1.8542536 0.3427193 0.3936326
H 1.9861678 -0.5491277 -1.0695765
H 1.6038476 -0.6826789 0.71771705
H -0.3440898 -2.149735 0.2746059
H -0.059199043 -2.0842772 -1.5339141

148
C -0.7241433 1.7412304 -0.5881256
O 0.03954617 0.5819761 

...

2910
O -0.16287966 -0.0606987 -1.9664239
C -0.20233507 -0.9542802 -1.1638639
C 0.18293808 -0.83368677 0.3203539
O 0.53423905 -1.8493112 0.8773291
C 0.1316937 0.4812835 1.0277603
C 0.4555734 1.7635076 0.35866812
O -0.8880261 1.3960402 0.6208708
H -0.4467087 -2.0038102 -1.4319932
H 0.28716752 0.3910666 2.1016963
H 0.85399425 2.563165 0.9812703
H 0.7395377 1.7639496 -0.6885525

2911
O -1.9206585 -0.9915136 -0.14454016
C -0.8535987 -1.3939197 0.23420465
C 0.44822347 -0.5838598 0.2027483
O 1.4296417 -1.0140169 0.7713084
N 0.404611 0.5605234 -0.5516164
C -0.46484643 1.7049354 -0.3858898
C 1.0252856 1.8192972 -0.21682517
H -0.70112836 -2.3931236 0.6913254
H -0.9377871 2.073547 -1.2908727
H -1.084136 1.7420424 0.50517046
H 1.4266653 1.9163326 0.78831285
H 1.6197164 2.2770476 -1.0016814

2912
C 0.10358285 1.9119815 0.64860797
C -0.08431049 0.38621694 0.5217571
C -1.5049925 0.031605564 0.5261858
C -2.6755686 -0.24265599 0.5126011
C 0.6133799 -0.16133457 -0.7190665
C 2.0875974 -0.47179592 -0.6747

H 1.466442 0.06638005 0.8660748
H 0.28093535 0.047381792 3.106969
H -1.3377359 0.26624027 2.2560515

2928
C 0.092447706 1.9977864 0.5030703
C -0.04990904 0.46953264 0.39846653
C -1.5191035 0.0888562 0.37926346
O -2.072148 -0.5976494 1.1985693
C 0.6189289 -0.072419874 -0.85657674
C 2.081112 -0.44235533 -0.83473116
C 1.0310459 -1.5207424 -0.9368829
H 1.1480337 2.2818742 0.46846658
H -0.4148328 2.497808 -0.33003092
H -0.3371249 2.3761144 1.4354761
H 0.38968524 -0.00957685 1.2803859
H -2.080384 0.5055667 -0.49276733
H 0.2552386 0.37395462 -1.7796109
H 2.6811635 -0.22017424 -1.7101719
H 2.6142015 -0.34266156 0.10528929
H 0.85633606 -2.1413481 -0.06428742
H 0.9127215 -2.0396347 -1.8814011

2929
C 0.2493536 2.1901667 -0.38277632
C 0.3799757 0.6687017 -0.53716195
C 1.2607617 0.093063 0.5658045
O 1.8032815 -0.98174834 0.5177921
C -0.9930731 -0.006491306 -0.5666036
C -1.177915 -1.4610769 -0.36380678
N -1.669457 -0.50269294 0.64682376
H 1.2293308 2.6780338 -0.3921852
H -0.3428092 2.61222 -1.19923

2944
C -1.0407637 2.2734427 -0.89472634
O -0.9757258 0.8429117 -0.8681685
C -0.047330122 0.3258721 -0.035646133
O 0.68722117 1.0003176 0.6477259
C -0.057438176 -1.15724 -0.07861941
C 1.2754688 -1.8623112 0.10924227
C 0.2718872 -1.8874372 1.2126707
H -0.08672408 2.698104 -1.2174996
H -1.8289737 2.5217943 -1.605727
H -1.2808356 2.6681995 0.0958497
H -0.7830175 -1.599553 -0.74860525
H 1.4407289 -2.7618346 -0.47214597
H 2.1387043 -1.2191685 0.23261495
H 0.46194687 -1.2611557 2.0761967
H -0.27068207 -2.8046799 1.4095212

2945
C -0.064872846 2.3537736 1.2434254
O -0.02936579 0.9196803 1.2581617
C 0.049948234 0.3339471 0.05252129
O 0.10280077 0.9362997 -0.9953689
C 0.07021333 -1.1519425 0.16724366
C -0.78669786 -1.8998291 -0.8048127
N 0.66306466 -1.8947799 -0.95525235
H 0.834477 2.7549503 0.77005464
H -0.9403097 2.710976 0.6954632
H -0.11690457 2.6590776 2.2882357
H 0.24970053 -1.554045 1.1575955
H -1.2214246 -2.8449652 -0.4935537
H -1.383614 -1.3062284 -1.4908788
H 0.9139109 -1.2104623 -1.66

2959
C -1.2196785 2.4253826 0.7810221
C -1.1816819 0.8977312 0.705598
C 0.08837372 0.35940173 0.033641677
O 1.2711308 0.8013832 0.70236915
C 0.06084901 -1.1493556 -0.09659304
C -0.27648216 -1.7870808 -1.3946621
N 1.0989757 -1.835185 -0.87564754
H -0.33249545 2.8054016 1.2938062
H -1.2395569 2.8682656 -0.22097684
H -2.1070814 2.7739282 1.3174822
H -1.2463714 0.4710674 1.7170435
H -2.0536108 0.5184076 0.15811221
H 0.17405754 0.7953391 -0.9713743
H 1.3105774 0.35413858 1.5554798
H -0.20978671 -1.6762528 0.8197306
H -0.7928762 -2.7425733 -1.4174831
H -0.4991886 -1.1346738 -2.2359338
H 1.6799419 -1.1412339 -1.3416867

2960
C -0.94850934 2.4042568 1.1775551
C -0.96744883 0.8844998 1.006724
C -0.31060135 0.42563853 -0.3049685
O 1.0311342 0.8714917 -0.40892318
C -0.36781466 -1.088966 -0.44444636
C 0.5993066 -1.8235935 -1.2633594
O 0.732623 -1.7774717 0.16789486
H 0.07605792 2.7807999 1.1249036
H -1.5255867 2.8972538 0.3869479
H -1.3781624 2.7002888 2.139072
H -0.44318664 0.40205973 1.8425122
H

2986
N 1.4186878 2.8538387 0.30396003
C 1.3112402 1.7271357 -0.2528686
O 0.39027438 0.844336 0.17903565
C 0.43091545 -0.4723642 -0.37349647
C -0.96435064 -1.0271866 -0.42690617
C -1.1658274 -2.4859884 -0.23296766
N -1.6130105 -1.5305283 0.7894616
H 2.1369083 3.4041624 -0.1620962
H 1.9022892 1.360643 -1.1050993
H 0.8703396 -0.44461465 -1.3806607
H 1.0692242 -1.10799 0.25707215
H -1.6372973 -0.48488164 -1.087349
H -1.9615928 -2.9970143 -0.76648927
H -0.2904723 -3.1003902 -0.034181137
H -0.9600157 -1.5093336 1.5695441

2987
N 1.0873052 2.644553 0.25976792
C 0.9238314 1.3945191 0.29113424
O 0.794513 0.71096337 -0.8704896
C 0.54351413 -0.6871057 -0.75667393
C -0.8911182 -0.9787839 -0.39970106
C -1.2531309 -2.1809695 0.36009812
O -1.2190573 -0.9008146 0.99179
H 1.1831521 3.002942 1.2077528
H 0.8721722 0.7576564 1.182184
H 0.7822466 -1.1046722 -1.7391647
H 1.2086611 -1.1447592 -0.012599699
H -1.6332138 -0.53067976 -1.060386
H -2.2424445 -2.616856 0.23455578
H -0.47544807 -2.877294 0.67179865
